In [1]:
%pip install streamlit
%pip install matplotlib
%pip install seaborn
%pip install scikit-learn
%pip install plotly
%pip install statsmodels

#Para instalar npm en visual studio
#1.Desde Google escribir node.js
#2. Instalar la versión más recomendada
! npm install localtunnel


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip3.12 in

In [12]:
%%writefile EVIDENCIA_2.py

## streamlit run EVIDENCIA_2.py

import os
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import streamlit as st

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import (
    r2_score, mean_absolute_error, mean_squared_error,
    precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix, roc_curve
)
from scipy.optimize import curve_fit

# =========================
# Utilidades base
# =========================
def safe_read_csv(path):
    try:
        if os.path.exists(path):
            return pd.read_csv(path)
    except Exception:
        pass
    return None

def unify_schema(df, city_name=None, verbose=False):
    """
    Normaliza nombres, resuelve columnas duplicadas, agrega 'city',
    y convierte strings con '%' a numéricos. Evita AttributeError.
    """
    if df is None or len(df) == 0:
        return None

    # 1) Normaliza nombres a snake_case
    cols_raw = list(df.columns)
    norm_cols = [str(c).strip().lower().replace(" ", "_") for c in cols_raw]

    # 2) Unifica duplicados: col, col__1, col__2, ...
    seen = {}
    unique_cols = []
    dups = []
    for c in norm_cols:
        if c not in seen:
            seen[c] = 0
            unique_cols.append(c)
        else:
            seen[c] += 1
            new_c = f"{c}__{seen[c]}"
            unique_cols.append(new_c)
            dups.append((c, new_c))
    if verbose and dups:
        print("Columnas duplicadas renombradas:", dups)

    df = df.copy()
    df.columns = unique_cols

    # 3) Correcciones habituales
    ren = {
        "accomodates": "accommodates",
        "bedroms": "bedrooms",
        "bathroom": "bathrooms",
        "host_response_time(%)": "host_response_rate",
        "review_scores_value": "review_scores_rating"
    }
    for k, v in ren.items():
        if k in df.columns and v not in df.columns:
            df.rename(columns={k: v}, inplace=True)

    # 4) Asegura 'city'
    if "city" not in df.columns:
        df["city"] = city_name if city_name else "unknown"
    else:
        if city_name:
            df["city"] = df["city"].fillna(city_name).replace("", city_name)

    # 5) Convierte columnas con símbolo %
    for c in df.columns:
        s = df[c]
        if pd.api.types.is_object_dtype(s):
            if s.astype(str).str.contains("%").any():
                s2 = s.astype(str).str.replace("%", "", regex=False)
                s2 = pd.to_numeric(s2, errors="coerce")
                df[c] = s2

    return df

def numeric_cols(df):
    return [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]

def categorical_cols(df):
    return [c for c in df.columns if (df[c].dtype == "object" or str(df[c].dtype).startswith("category"))]

def r_from_y_ypred(y_true, y_pred):
    if len(y_true) < 2 or len(y_pred) < 2:
        return np.nan
    try:
        return float(np.corrcoef(y_true, y_pred)[0,1])
    except Exception:
        return np.nan

def fit_nonlinear_series(x, y, model_name):
    def f_quad(z, a, b, c): return a*z**2 + b*z + c
    def f_exp(z, a, b, c):  return a*np.exp(b*z) + c
    def f_inv(z, a, b):     return a + b/np.where(z==0, np.finfo(float).eps, z)

    if model_name == "Cuadrática":
        popt, _ = curve_fit(f_quad, x, y, maxfev=20000)
        yhat = f_quad(x, *popt)
        return yhat, popt
    elif model_name == "Exponencial":
        p0 = (1.0, 0.01, float(np.nanmedian(y)))
        popt, _ = curve_fit(f_exp, x, y, p0=p0, maxfev=20000)
        yhat = f_exp(x, *popt)
        return yhat, popt
    else:  # Inversa
        popt, _ = curve_fit(f_inv, x, y, maxfev=20000)
        yhat = f_inv(x, *popt)
        return yhat, popt

def coerce_numeric_cols(df, cols):
    """Devuelve una copia donde 'cols' se convierten a numérico con errors='coerce'."""
    d = df.copy()
    for c in cols:
        if c in d.columns:
            d[c] = pd.to_numeric(d[c], errors='coerce')
    return d

# =========================
# Configuración app
# =========================
st.set_page_config(
    page_title="Dashboard Mult-ciudad Airbnb: Proyecto Innovativo Grupo IPYNB",
    page_icon="🏠",
    layout="wide"
)

# --- Portada (Imagen + Título + Descripción) ---
# Ruta proporcionada por el usuario (cámbiala si tu archivo está en otra ubicación)
PORTADA_PATHS = [
    "WhatsApp Image 2025-10-22 at 11.38.03 PM.jpeg",
    "./assets/aura.jpg",
    "./assets/aura.png",
    "/mnt/data/WhatsApp Image 2025-10-22 at 11.38.03 PM.jpeg",
]
img_path = next((p for p in PORTADA_PATHS if os.path.exists(p)), None)
if img_path:
    col_l, col_c, col_r = st.columns([1,2,1])
    with col_c:
        st.image(img_path, use_column_width=True)
else:
    st.info("⚠️ Imagen de portada no encontrada. Verifica la ruta en PORTADA_PATHS.")

st.title("Dashboard Mult-ciudad Airbnb: Proyecto Innovativo Grupo IPYNB")
st.write("**Este dashboard permite establecer un panorama completo de Airbnb, sus Hosts y Alojamientos, potencializando al Programa AURA.**")

# =========================
# Sidebar
# =========================
with st.sidebar:
    st.header("⚙️ Configuración")
    color = st.color_picker("Color principal", "#0B6E4F")
    grid_alpha = st.slider("Transparencia de rejilla", 0.0, 1.0, 0.2, 0.05)
    st.markdown("---")
    st.subheader("📂 Carga de datos (opcional)")
    ups = st.file_uploader("Sube CSV (puedes múltiples)", type=["csv"], accept_multiple_files=True)

# =========================
# Carga de datasets base
# =========================
df_mex = unify_schema(safe_read_csv("Mexico_Limpio.csv"), "Mexico")
df_mad = unify_schema(safe_read_csv("Madrid_AirBnb_010.csv"), "Madrid")
df_val = unify_schema(safe_read_csv("VALENCIA_LIMPIO.csv"), "Valencia")
df_rio = unify_schema(safe_read_csv("base_final.csv"), "Rio")   # <- etiqueta "Rio"
df_roma = unify_schema(safe_read_csv("df_limp_ROMA.csv"), "Roma")

dfs_raw = [d for d in [df_mex, df_mad, df_val, df_rio, df_roma] if d is not None]

if ups:
    for f in ups:
        try:
            tmp = pd.read_csv(f)
            dfs_raw.append(unify_schema(tmp, city_name=os.path.splitext(f.name)[0]))
        except Exception as e:
            st.warning(f"No se pudo leer {f.name}: {e}")

if not dfs_raw:
    st.error("No se encontraron datos. Coloca los CSV en la carpeta o súbelos.")
    st.stop()

# Intersección de columnas comunes para asegurar comparabilidad
common_cols = set(dfs_raw[0].columns)
for d in dfs_raw[1:]:
    common_cols = common_cols.intersection(set(d.columns))
common_cols = sorted(list(common_cols))

# Reindexa cada DF a columnas comunes
dfs = [d[common_cols].copy() for d in dfs_raw]

# Concat para análisis global
df_all = pd.concat(dfs, ignore_index=True)
cities_all = sorted(df_all["city"].dropna().astype(str).unique().tolist())

# Selección de ciudades a comparar
sel_cities = st.multiselect("🌍 Ciudades a comparar", options=cities_all, default=cities_all)
if not sel_cities:
    st.warning("Selecciona al menos una ciudad.")
    st.stop()
df = df_all[df_all["city"].isin(sel_cities)].reset_index(drop=True)

# Columnas por tipo (en universo común)
num_cols_all = [c for c in common_cols if pd.api.types.is_numeric_dtype(df[c])]
cat_cols_all = [c for c in common_cols if (df[c].dtype == "object" or str(df[c].dtype).startswith("category"))]

# =========================
# Tabs (7 secciones)
# =========================
tabs = st.tabs([
    "1) 🧰 Extracción",
    "2) 🔠 Categóricas (10 comunes)",
    "3) 📈 Regresión Lineal Simple",
    "4) 🧮 Regresión Lineal Múltiple",
    "5) 🌀 Regresión No Lineal",
    "6) 🧪 Regresión Logística",
    "7) 💡 Insights & Propuestas",
])

# ============================================================
# TAB 1 - Extracción (comparativos de variables de servicio)
# ============================================================
with tabs[0]:
    st.subheader("🧰 Extracción: comparativo de variables clave por ciudad")
    st.caption("Variables típicas: price, number_of_reviews, availability_365, calculated_host_listings_count, accommodates, reviews_per_month, minimum_nights, estimated_occupancy_l365d, review_scores_rating.")

    candidates = [
        "price", "number_of_reviews", "availability_365",
        "calculated_host_listings_count", "host_total_listings_count",
        "accommodates", "reviews_per_month", "minimum_nights",
        "estimated_occupancy_l365d", "review_scores_rating"
    ]
    present = [c for c in candidates if c in common_cols]
    sel_vars = st.multiselect("Variables a comparar", options=present, default=present)

    if not sel_vars:
        st.info("Selecciona al menos una variable.")
    else:
        # Conversión segura a numérico
        df_num = coerce_numeric_cols(df, sel_vars)

        # KPIs por ciudad (mediana y promedio)
        try:
            kpi = (df_num.groupby("city")[sel_vars]
                   .agg(['median', 'mean'])
                   .round(3))
            st.dataframe(kpi, use_container_width=True)
        except Exception as e:
            st.warning(f"No fue posible calcular median/mean: {e}")

        # Gráficas por variable (box + barras) con columnas numéricas
        for v in sel_vars:
            st.markdown(f"**Variable:** `{v}`")
            c1, c2 = st.columns(2)

            with c1:
                try:
                    fig = px.box(df_num, x="city", y=v, points="outliers", color="city",
                                 title=f"Distribución de {v} por ciudad")
                    st.plotly_chart(fig, use_container_width=True)
                except Exception:
                    st.info(f"No se pudo graficar boxplot para {v}.")

            with c2:
                try:
                    # SeriesGroupBy.median sin numeric_only (ya es numérico)
                    agg = df_num.groupby("city")[v].median().reset_index()
                    fig2 = px.bar(agg, x="city", y=v, title=f"Mediana de {v} por ciudad")
                    st.plotly_chart(fig2, use_container_width=True)
                except Exception:
                    st.info(f"No se pudo graficar barras para {v}.")

# ============================================================
# TAB 2 - Categóricas (10 comunes)
# ============================================================
with tabs[1]:
    st.subheader("🔠 Análisis de variables categóricas (10 comunes)")
    cat_common = [c for c in cat_cols_all if c not in ["city"]]
    sel_cats = st.multiselect("Selecciona hasta 10 categóricas", options=cat_common, default=cat_common[:10], max_selections=10)
    if not sel_cats:
        st.info("Selecciona al menos una variable categórica (máx. 10).")
    else:
        for cat in sel_cats:
            st.markdown(f"**Categórica:** `{cat}`")
            # Distribución global
            dist = df[cat].astype("category").value_counts(dropna=False).reset_index()
            dist.columns = [cat, "freq"]
            c1, c2 = st.columns(2)
            with c1:
                try:
                    fig = px.bar(dist, x=cat, y="freq", title=f"Frecuencia global de {cat}")
                    st.plotly_chart(fig, use_container_width=True)
                except Exception:
                    st.dataframe(dist)
            with c2:
                try:
                    fig2 = px.pie(dist, names=cat, values="freq", title=f"Participación de {cat}")
                    st.plotly_chart(fig2, use_container_width=True)
                except Exception:
                    st.dataframe(dist)

            # Comparativo por ciudad
            try:
                cross = (df.groupby(["city", cat]).size()
                         .reset_index(name="n"))
                fig3 = px.bar(cross, x="city", y="n", color=cat, barmode="group",
                              title=f"{cat} por ciudad")
                st.plotly_chart(fig3, use_container_width=True)
            except Exception:
                st.info(f"No se pudo graficar comparativo para {cat}.")

# ============================================================
# TAB 3 - Regresión Lineal Simple (por ciudad)
# ============================================================
with tabs[2]:
    st.subheader("📈 Regresión Lineal Simple (comparativa por ciudad)")
    if len(num_cols_all) < 2:
        st.info("Se requieren al menos dos variables numéricas comunes.")
    else:
        col1, col2 = st.columns(2)
        with col1:
            y_lin = st.selectbox("Y (numérica)", options=num_cols_all, key="lin_y")
        with col2:
            x_lin = st.selectbox("X (numérica)", options=[c for c in num_cols_all if c != y_lin], key="lin_x")

        sel_c_lin = st.multiselect("Ciudades a incluir", options=sel_cities, default=sel_cities, key="lin_cities")
        results = []
        for city in sel_c_lin:
            dfx = df[df["city"]==city][[x_lin, y_lin]].copy()
            dfx[x_lin] = pd.to_numeric(dfx[x_lin], errors='coerce')
            dfx[y_lin] = pd.to_numeric(dfx[y_lin], errors='coerce')
            dfx = dfx.dropna()
            if len(dfx) < 8:
                results.append({"city": city, "R2": np.nan, "R": np.nan, "n": len(dfx)})
                st.info(f"{city}: datos insuficientes.")
                continue
            X = dfx[[x_lin]].values
            Y = dfx[y_lin].values
            Xtr, Xte, Ytr, Yte = train_test_split(X, Y, test_size=0.3, random_state=42)
            model = LinearRegression().fit(Xtr, Ytr)
            yhat = model.predict(Xte)
            R2 = r2_score(Yte, yhat)
            R = r_from_y_ypred(Yte, yhat)
            results.append({"city": city, "R2": R2, "R": R, "n": len(dfx)})

            # Gráfico por ciudad
            fig, ax = plt.subplots()
            ax.scatter(X, Y, alpha=0.5)
            X_sorted = np.sort(X, axis=0)
            ax.plot(X_sorted, model.predict(X_sorted), color=color, lw=2)
            ax.set_title(f"{city} | {y_lin} ~ {x_lin}")
            ax.set_xlabel(x_lin); ax.set_ylabel(y_lin); ax.grid(alpha=grid_alpha)
            st.pyplot(fig, use_container_width=True)

        st.markdown("**Resumen de métricas (test):**")
        st.dataframe(pd.DataFrame(results).round(3), use_container_width=True)

# ============================================================
# TAB 4 - Regresión Lineal Múltiple (por ciudad)
# ============================================================
with tabs[3]:
    st.subheader("🧮 Regresión Lineal Múltiple (comparativa por ciudad)")
    if len(num_cols_all) < 3:
        st.info("Se requieren ≥ 3 variables numéricas comunes para múltiple.")
    else:
        col1, col2 = st.columns([1,2])
        with col1:
            y_mul = st.selectbox("Y (numérica)", options=num_cols_all, key="mul_y")
        with col2:
            x_mul = st.multiselect("X (numéricas)", options=[c for c in num_cols_all if c != y_mul],
                                   default=[c for c in num_cols_all if c != y_mul][:3], key="mul_x")
        if not x_mul:
            st.info("Selecciona al menos una X.")
        else:
            sel_c_mul = st.multiselect("Ciudades a incluir", options=sel_cities, default=sel_cities, key="mul_cities")
            rows = []
            for city in sel_c_mul:
                cols = [y_mul] + x_mul
                dfx = df[df["city"]==city][cols].copy()
                for c in cols:
                    dfx[c] = pd.to_numeric(dfx[c], errors='coerce')
                dfx = dfx.dropna()
                if len(dfx) < 10:
                    rows.append({"city": city, "R2": np.nan, "R": np.nan, "n": len(dfx)})
                    st.info(f"{city}: datos insuficientes.")
                    continue
                X = dfx[x_mul].values
                Y = dfx[y_mul].values
                Xtr, Xte, Ytr, Yte = train_test_split(X, Y, test_size=0.3, random_state=42)
                model = LinearRegression().fit(Xtr, Ytr)
                yhat = model.predict(Xte)
                R2 = r2_score(Yte, yhat)
                R = r_from_y_ypred(Yte, yhat)
                rows.append({"city": city, "R2": R2, "R": R, "n": len(dfx)})

                # Gráfico real vs predicho
                fig, ax = plt.subplots()
                ax.scatter(Yte, yhat, alpha=0.6)
                m = [min(Yte.min(), yhat.min()), max(Yte.max(), yhat.max())]
                ax.plot(m, m, '--', lw=1)
                ax.set_title(f"{city} | {y_mul} ~ {', '.join(x_mul)}")
                ax.set_xlabel("Real"); ax.set_ylabel("Predicho"); ax.grid(alpha=grid_alpha)
                st.pyplot(fig, use_container_width=True)

            st.markdown("**Resumen de métricas (test):**")
            st.dataframe(pd.DataFrame(rows).round(3), use_container_width=True)

# ============================================================
# TAB 5 - Regresión No Lineal (por ciudad)
# ============================================================
with tabs[4]:
    st.subheader("🌀 Regresión No Lineal (comparativa por ciudad)")
    if len(num_cols_all) < 2:
        st.info("Se requieren al menos dos variables numéricas comunes.")
    else:
        c1, c2, c3 = st.columns(3)
        with c1:
            y_nl = st.selectbox("Y", options=num_cols_all, key="nl_y")
        with c2:
            x_nl = st.selectbox("X", options=[c for c in num_cols_all if c != y_nl], key="nl_x")
        with c3:
            model_nl = st.selectbox("Modelo", ["Cuadrática", "Exponencial", "Inversa"], index=0)

        sel_c_nl = st.multiselect("Ciudades a incluir", options=sel_cities, default=sel_cities, key="nl_cities")
        rows = []
        for city in sel_c_nl:
            dfx = df[df["city"]==city][[x_nl, y_nl]].copy()
            dfx[x_nl] = pd.to_numeric(dfx[x_nl], errors='coerce')
            dfx[y_nl] = pd.to_numeric(dfx[y_nl], errors='coerce')
            dfx = dfx.dropna().sort_values(x_nl)
            if len(dfx) < 10:
                rows.append({"city": city, "R2": np.nan, "R": np.nan, "n": len(dfx)})
                st.info(f"{city}: datos insuficientes.")
                continue
            x = dfx[x_nl].astype(float).values
            y = dfx[y_nl].astype(float).values
            try:
                yhat, popt = fit_nonlinear_series(x, y, model_nl)
                R2 = r2_score(y, yhat)
                R = r_from_y_ypred(y, yhat)
                rows.append({"city": city, "R2": R2, "R": R, "n": len(dfx)})

                # Gráfico
                fig, ax = plt.subplots()
                ax.scatter(x, y, alpha=0.5)
                ax.plot(x, yhat, color=color, lw=2)
                ax.set_title(f"{city} | {y_nl} ~ {x_nl} ({model_nl})")
                ax.set_xlabel(x_nl); ax.set_ylabel(y_nl); ax.grid(alpha=grid_alpha)
                st.pyplot(fig, use_container_width=True)
            except Exception as e:
                rows.append({"city": city, "R2": np.nan, "R": np.nan, "n": len(dfx)})
                st.warning(f"{city}: fallo del ajuste ({e}).")

        st.markdown("**Resumen de métricas (ajuste in-sample):**")
        st.dataframe(pd.DataFrame(rows).round(3), use_container_width=True)

# ============================================================
# TAB 6 - Regresión Logística (por ciudad)
# ============================================================
with tabs[5]:
    st.subheader("🧪 Regresión Logística (comparativa por ciudad)")
    cols_all = list(common_cols)
    y_choice = st.selectbox("Variable dependiente (Y)", options=cols_all, key="log_y")
    x_choices = st.multiselect("Variables explicativas (X)", options=[c for c in cols_all if c != y_choice], max_selections=6, key="log_xs")

    if not x_choices:
        st.info("Selecciona al menos una X.")
    else:
        dfY = df[y_choice]
        # --- Binarización Y (SOLO Media o Manual) ---
        st.markdown("**Binarización de Y**")
        if pd.api.types.is_numeric_dtype(dfY):
            howY = st.selectbox("Método (Y numérica)", ["Media", "Manual"], index=0, key="y_thr_m")
            if howY == "Media":
                thrY = float(dfY.dropna().mean())
            else:
                thrY = st.number_input("Umbral manual Y", value=float(dfY.dropna().mean()), key="y_thr_i")
            y_bin = (dfY >= thrY).astype(int)
            st.caption(f"Y binaria: 1 si {y_choice} ≥ {thrY:.4f}")
        else:
            uniq = dfY.dropna().astype(str).unique().tolist()
            pos = st.selectbox("Clase positiva (Y=1)", options=sorted(uniq), key="y_pos")
            y_bin = (dfY.astype(str) == pos).astype(int)
            st.caption(f"Y binaria: 1 si {y_choice} == '{pos}'")

        # --- Binarización opcional de UNA X numérica (SOLO Media o Manual) ---
        st.markdown("**Binarización opcional de X (una X numérica específica)**")
        binarize_x = st.checkbox("Binarizar una X numérica", value=False, key="bx_flag")
        X_design = df[x_choices].copy()
        if binarize_x:
            x_num_candidates = [c for c in x_choices if pd.api.types.is_numeric_dtype(df[c])]
            if x_num_candidates:
                x_bin_name = st.selectbox("X a binarizar", options=x_num_candidates, key="bx_sel")
                howX = st.selectbox("Método (X numérica)", ["Media", "Manual"], index=0, key="x_thr_m")
                if howX == "Media":
                    x_bin_thr = float(df[x_bin_name].dropna().mean())
                else:
                    x_bin_thr = st.number_input("Umbral manual X", value=float(df[x_bin_name].dropna().mean()), key="x_thr_i")
                X_design[x_bin_name] = (df[x_bin_name] >= x_bin_thr).astype(int)
                st.caption(f"{x_bin_name} binaria: 1 si ≥ {x_bin_thr:.4f}")
            else:
                st.info("No hay X numéricas seleccionadas para binarizar.")

        # --- class_weight & split ---
        use_bal = st.checkbox("Usar class_weight='balanced'", value=True, key="cw")
        test_size = st.slider("Proporción de test", 0.1, 0.5, 0.3, 0.05, key="tsz")

        sel_c_log = st.multiselect("Ciudades a incluir", options=sel_cities, default=sel_cities, key="log_cities")
        rows = []
        for city in sel_c_log:
            dfx = X_design[df["city"]==city].copy()
            yy = y_bin[df["city"]==city].copy()
            # Conversión numérica de X por seguridad
            for c in dfx.columns:
                dfx[c] = pd.to_numeric(dfx[c], errors='coerce')
            data = pd.concat([dfx, yy.rename("__y")], axis=1).dropna()
            if data["__y"].nunique() < 2 or len(data) < 30:
                rows.append({"city": city, "AUC": np.nan, "Precision": np.nan, "Recall": np.nan, "F1": np.nan, "n": len(data)})
                st.info(f"{city}: datos insuficientes o una sola clase.")
                continue

            X = data.drop(columns="__y").values
            y = data["__y"].astype(int).values

            Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=test_size, random_state=42, stratify=y)
            pipe = Pipeline([
                ("scaler", StandardScaler(with_mean=True, with_std=True)),
                ("logit", LogisticRegression(max_iter=600, class_weight=("balanced" if use_bal else None)))
            ])
            pipe.fit(Xtr, ytr)
            proba = pipe.predict_proba(Xte)[:,1]
            pred = (proba >= 0.5).astype(int)

            AUC = roc_auc_score(yte, proba)
            PREC = precision_score(yte, pred, zero_division=0)
            REC = recall_score(yte, pred, zero_division=0)
            F1 = f1_score(yte, pred, zero_division=0)
            rows.append({"city": city, "AUC": AUC, "Precision": PREC, "Recall": REC, "F1": F1, "n": len(data)})

            # Gráfico ROC por ciudad
            fpr, tpr, thr = roc_curve(yte, proba)
            fig, ax = plt.subplots()
            ax.plot(fpr, tpr, lw=2, label=f"AUC={AUC:.3f}")
            ax.plot([0,1],[0,1],'--', lw=1)
            ax.set_title(f"ROC - {city}")
            ax.set_xlabel("FPR"); ax.set_ylabel("TPR"); ax.legend(); ax.grid(alpha=grid_alpha)
            st.pyplot(fig, use_container_width=True)

        st.markdown("**Resumen de métricas (test):**")
        st.dataframe(pd.DataFrame(rows).round(3), use_container_width=True)

# ============================================================
# TAB 7 - Insights & Propuestas
# ============================================================
with tabs[6]:
    st.subheader("💡 Hallazgos e Insights accionables")
    st.caption("Hasta 6 insights (3 sobre hosts, 3 sobre hospedajes), basados en las ciudades seleccionadas y columnas disponibles.")

    notes_hosts = []
    notes_stays = []

    # --------- HOSTS ----------
    if "host_is_superhost" in df.columns:
        sup_series = df["host_is_superhost"]
        if sup_series.dtype == "object":
            sup_num = sup_series.astype(str).str.lower().isin(["t","true","1","yes","si"]).astype(float)
        else:
            sup_num = pd.to_numeric(sup_series, errors='coerce')
        aux = sup_num.groupby(df["city"]).mean().reset_index(name="superhost_rate")
        aux["superhost_rate"] = (aux["superhost_rate"]*100).round(1)
        st.plotly_chart(px.bar(aux, x="city", y="superhost_rate", title="% de superhosts por ciudad"),
                        use_container_width=True)
        top_city = aux.sort_values("superhost_rate", ascending=False).iloc[0]
        notes_hosts.append(f"Mayor % de superhosts en **{top_city['city']}** ({top_city['superhost_rate']}%).")

    if "host_response_rate" in df.columns:
        rr = pd.to_numeric(df["host_response_rate"], errors="coerce")
        aux = rr.groupby(df["city"]).mean().reset_index(name="host_response_rate")
        aux["host_response_rate"] = aux["host_response_rate"].round(1)
        st.plotly_chart(px.bar(aux, x="city", y="host_response_rate", title="Tasa de respuesta promedio del host (%)"),
                        use_container_width=True)
        best = aux.sort_values("host_response_rate", ascending=False).iloc[0]
        notes_hosts.append(f"Mejor tasa de respuesta en **{best['city']}** ({best['host_response_rate']}%).")

    for col in ["host_total_listings_count", "calculated_host_listings_count"]:
        if col in df.columns:
            col_num = pd.to_numeric(df[col], errors="coerce")
            aux = col_num.groupby(df["city"]).mean().reset_index(name=col)
            aux[col] = aux[col].round(2)
            st.plotly_chart(px.bar(aux, x="city", y=col, title=f"Promedio de listados por host ({col})"),
                            use_container_width=True)
            heavy = aux.sort_values(col, ascending=False).iloc[0]
            notes_hosts.append(f"Mayor concentración de listados por host en **{heavy['city']}** ({heavy[col]} en promedio).")
            break

    # --------- HOSPEDAJES ----------
    if all(c in df.columns for c in ["price","room_type"]):
        price_num = pd.to_numeric(df["price"], errors="coerce")
        aux = price_num.groupby([df["city"], df["room_type"]]).median().reset_index(name="price")
        st.plotly_chart(px.bar(aux, x="city", y="price", color="room_type",
                               barmode="group", title="Precio mediano por tipo de habitación y ciudad"),
                        use_container_width=True)
        top = aux.sort_values("price", ascending=False).head(1).iloc[0]
        notes_stays.append(f"Precio mediano más alto: **{top['city']}**, **{top['room_type']}** (~{top['price']:.0f}).")

    if "estimated_occupancy_l365d" in df.columns:
        occ_num = pd.to_numeric(df["estimated_occupancy_l365d"], errors="coerce")
        aux = occ_num.groupby(df["city"]).mean().reset_index(name="estimated_occupancy_l365d")
        aux["estimated_occupancy_l365d"] = aux["estimated_occupancy_l365d"].round(2)
        st.plotly_chart(px.bar(aux, x="city", y="estimated_occupancy_l365d", title="Ocupación estimada (últimos 365 días)"),
                        use_container_width=True)
        best = aux.sort_values("estimated_occupancy_l365d", ascending=False).iloc[0]
        notes_stays.append(f"Mayor ocupación estimada en **{best['city']}** ({best['estimated_occupancy_l365d']}).")

    if "review_scores_rating" in df.columns:
        rs_num = pd.to_numeric(df["review_scores_rating"], errors="coerce")
        aux = rs_num.groupby(df["city"]).mean().reset_index(name="review_scores_rating")
        aux["review_scores_rating"] = aux["review_scores_rating"].round(2)
        st.plotly_chart(px.bar(aux, x="city", y="review_scores_rating", title="Puntaje promedio de reseñas"),
                        use_container_width=True)
        best = aux.sort_values("review_scores_rating", ascending=False).iloc[0]
        notes_stays.append(f"Mejor calificación promedio en **{best['city']}** ({best['review_scores_rating']}).")

    st.markdown("### 📝 Resumen de hallazgos")
    if not (notes_hosts or notes_stays):
        st.info("No se pudieron generar insights (faltan columnas clave).")
    else:
        insights = notes_hosts[:3] + notes_stays[:3]
        for i, n in enumerate(insights, 1):
            st.markdown(f"**{i}.** {n}")

    st.markdown("---")
    st.subheader("🛠️ Propuestas de mejora basadas en los hallazgos")

    props_hosts = [
        "Programa **Respuesta < 1h**: bonificar visibilidad a anfitriones con tasa de respuesta >95% y tiempo <60 min (plantillas de respuesta + alertas).",
        "**Gestión de cartera de listings**: en ciudades con alta concentración por host, aplicar pricing dinámico con límites y mínimos de estadía para reducir canibalización.",
        "**Ruta a Superhost**: checklist de amenities críticos + auditorías ligeras trimestrales cuando el puntaje de reseñas esté debajo de la media de ciudad."
    ]
    props_stays = [
        "**Bundles por tipo de habitación** (late checkout, limpieza premium) para elevar RevPAR en ciudades con precio mediano alto.",
        "**Playbook de recuperación de reseñas**: mensaje proactivo pre-llegada, check-in asistido y encuesta 'in-stay' (NPS breve).",
        "**Empuje de ocupación** en ciudades con baja ocupación estimada: campañas geotarget por estancias medias (7–14 días) + descuentos escalonados + políticas flexibles en temporada baja."
    ]

    st.markdown("**Hosts**")
    for p in props_hosts:
        st.markdown(f"- {p}")

    st.markdown("**Hospedajes**")
    for p in props_stays:
        st.markdown(f"- {p}")

Overwriting EVIDENCIA_2.py
